In [1]:
from pandas_datareader import data
from yahoo_fin import stock_info as si
import pandas as pd
from tqdm import tqdm 

In [2]:
df1 = pd.DataFrame(si.tickers_sp500())
df2 = pd.DataFrame(si.tickers_nasdaq())
df3 = pd.DataFrame(si.tickers_dow())
df4 = pd.DataFrame(si.tickers_other())

In [3]:
sym1 = set(symbol for symbol in df1[0].values.tolist())
sym2 = set(symbol for symbol in df2[0].values.tolist())
sym3 = set(symbol for symbol in df3[0].values.tolist())
sym4 = set(symbol for symbol in df4[0].values.tolist())

In [4]:
# join the 4 sets into one. Because it's a set, there will be no duplicate symbols
symbols = set.union(sym1, sym2, sym3, sym4)

# Some stocks are 5 characters. Those stocks with the suffixes listed below are not of interest.
my_list = ['M', 'W', 'R', 'P', 'Q', 'U', 'B', 'D', 'L','I']
del_set = set()
sav_set = set()

for symbol in symbols:
    if len(symbol) >= 5:
        del_set.add(symbol)
    else:
        sav_set.add(symbol)

print(f'Removed {len( del_set )} unqualified stock symbols...' )
print(f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1692 unqualified stock symbols...
There are 9635 qualified stock symbols...


In [6]:
import yahooquery as yq

results = {} 

errors = []
for ticker in tqdm(list(sav_set)[1:]): 
    if "$" in ticker or "-" in ticker or "." in ticker: 
        continue
    yq_ticker = yq.Ticker(ticker)
    mc = yq_ticker.price[ticker]['marketCap']
    if mc == {}: 
        errors.append(ticker)
        continue
    if mc > 1000000000: 
        results[ticker] = {'Market Cap' : mc,
                           'Name' : yq_ticker.price[ticker]['longName']}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9634/9634 [20:41<00:00,  7.76it/s]


In [11]:
df = pd.DataFrame.from_dict(results, orient = 'columns')

In [12]:
df.to_csv('companies.csv') 

In [15]:
df.T.to_csv('companies.csv')